In [59]:
import os
import pandas as pd
import math

In [60]:
# Load the annotation data
matrix_anno = pd.read_csv("../../data/csv.data/E-MTAB-6149_NSCLC_meta.csv")
matrix_anno = matrix_anno.rename(columns={"Unnamed: 0": "cellName"})

In [61]:
# Process the annotation data

# remove "macrophage other", "malignant"
matrix_anno.drop(matrix_anno[matrix_anno["uniformCellTypeSub"] == "Malignant"].index, inplace=True)
matrix_anno.drop(matrix_anno[matrix_anno["uniformCellTypeSub"] == "Macrophage other"].index, inplace=True)

# make plasma subtype of B cell
matrix_anno.loc[matrix_anno["uniformCellTypeSub"] == "Plasma", "uniformCellType"] = "B cell"

anno_celltype = matrix_anno.loc[:, ["cellName", "uniformCellType", "uniformCellTypeSub"]]


In [62]:
os.getcwd()
# matrix_data = pd.read_csv("../../data/csv.data/E-MTAB-6149_NSCLC_TPM.csv")
matrix_data = pd.read_csv("../../data/csv.data/test1000gene.csv")

In [63]:
print(matrix_data.shape)

(1000, 39324)


In [64]:

matrix_data = matrix_data.rename(columns={"Unnamed: 0": "cellName"})
matrix_transpose = matrix_data.T
cellName = matrix_transpose.index.tolist()

# rename the col name for matrix_data.T
matrix_transpose["cellName"] = cellName
matrix_transpose.columns = matrix_transpose.iloc[0, :]
matrix_transpose.drop(index= "cellName", inplace=True)

In [80]:
# merge matrix with anno
merge_df = pd.merge(matrix_transpose, anno_celltype, on="cellName", how="inner")

# group by subtype. calculate mean expression
group_subtype_mean = merge_df.groupby("uniformCellTypeSub").mean()

# ####NO longer need to calc mean for Major cell type. 

# drop duplicated row
group_subtype_mean = group_subtype_mean.drop_duplicates()

allCellType = group_subtype_mean

# rescale
allCellType_reScale = allCellType.applymap(lambda x: math.log2(x/10 + 1))

# 2 decimals
allCellType_reScale= allCellType_reScale.applymap(lambda x: format(float(x), ".2f"))

print(allCellType_reScale)

/Users/_alanglanglang/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3460: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


                           SAMD11 NOC2L KLHL17 PLEKHN1 PERM1  HES4 ISG15  \
uniformCellTypeSub                                                         
B cell exhausted             0.00  2.75   0.00    0.04  0.00  0.41  5.46   
B cell naive                 0.00  2.98   0.00    0.15  0.00  0.00  4.20   
B cell non-switched memory   0.00  2.92   0.28    0.00  0.00  0.33  4.89   
B cell switched memory       0.12  2.12   0.16    0.21  0.00  0.68  5.11   
CAF                          1.70  2.68   0.44    0.41  0.09  1.47  5.39   
Endothelial                  0.08  2.61   0.85    0.43  0.00  0.76  5.32   
Macrophage M1                0.03  1.48   0.05    0.32  0.00  0.96  8.99   
Macrophage M2                0.04  1.67   0.25    0.10  0.01  0.14  6.82   
Mast                         0.00  1.99   0.00    0.26  0.00  0.63  4.65   
NK                           0.00  2.74   0.12    0.18  0.00  0.47  4.81   
Plasma                       0.62  2.85   0.41    1.00  0.00  1.11  6.65   
T CD4 naive 

In [ ]:
print(group_subtype_mean)

In [ ]:
# For each gene, score the expression level for each celltype. (top  10% ==1, else, 0)

### test with 3 genes
### mean_expr_celltype = pd.read_csv("../../data/mean.exp.reScale.csv")
mean_expr_celltype = allCellType_reScale

scoring_df = pd.DataFrame()
scoring_df.index = mean_expr_celltype.iloc[:,0]

# assign 1/0 new label for all cell type to indicate high/low expr level
gene_list = mean_expr_celltype.columns.tolist()

for gene in gene_list:
    mean_expr_celltype[gene] = mean_expr_celltype[gene].map(lambda x: float(x))
    top20quantl = mean_expr_celltype[gene].quantile(.2)
    mean_expr_celltype.loc["top20quantl", gene]  = top20quantl

    median = mean_expr_celltype[gene].median()
    mean_expr_celltype[gene] = mean_expr_celltype[gene].map(lambda x :1 if (x > median and x > 1)else 0 )

    # mean_expr_celltype[gene] = mean_expr_celltype[gene].map(lambda x :1 if x > 0.5 else 0 )

###  drop added parent node
# group_subtype_mean = group_subtype_mean.iloc[:25,:]

print(mean_expr_celltype)


In [101]:
# Non-lymphoid branch celltypes 
non_lymph = ["CAF","Endothelial", "Macrophage M1","Macrophage M2",
              "Mast","Plasma", "cDC1 CLEC9A", "cDC2 CD1C", 
              "cDC3 LAMP3","pDC"]

non_lymph_df = mean_expr_celltype.loc[non_lymph,:]
lymph_df = mean_expr_celltype.drop(index = non_lymph)

# non_lymph_df.loc["sum"] = non_lymph_df.sum()
# lymph_df.loc["sum"] = lymph_df.sum()

two_branch = pd.DataFrame({"non_lymph_sum":non_lymph_df.sum(),
                            "lymph_sum":lymph_df.sum()
                           })
two_branch.columns = ["non_lymph_sum", "lymph_sum"]
print(two_branch.columns)

two_branch["exprBoth"] =  two_branch.apply(lambda x: 1 \
                                       if (x['non_lymph_sum'] * x['lymph_sum'] != 0 \
                                          and x['non_lymph_sum'] + x['lymph_sum']>12.5) else 0, axis=1)

# x["non_lymph_sum"] + x["lymph_sum"]
both_branch = two_branch[two_branch["exprBoth"]== 1]
print(both_branch)
print(both_branch.shape)
# two_branch.to_csv("output/two_branch.csv")
both_branch.to_csv("output/both_branch_more12.csv")

Index(['non_lymph_sum', 'lymph_sum'], dtype='object')
          non_lymph_sum  lymph_sum  exprBoth
NOC2L                 4          9         1
ISG15                 6          7         1
TNFRSF18              2         11         1
TNFRSF4               4          9         1
SDF4                  4          9         1
...                 ...        ...       ...
PEX11B                6          7         1
RBM8A                 3         10         1
LIX1L                 3         10         1
POLR3GL               1         12         1
PRKAB2                3         10         1

[335 rows x 3 columns]
(335, 3)
